In [1]:
from dotenv import load_dotenv

from pathlib import Path
import os 
import sys
import numpy as np
import json

load_dotenv()
GEMINI_KEY = os.getenv('GEMINI_KEY')

root = Path().resolve().parent

root_str = str(root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)

if Path.cwd() != root:
    os.chdir(root)

# Results

In [2]:
import pandas as pd
import json
from pprint import pprint

results = json.load(open("results/bench_results.json"))

reshuffle = {}
for metric in ["is_valid", "correctness", "compliance"]:
    reshuffle[metric] = {
        model: {
            task: results[model][task][metric]
            for task in results[model]
        }
        for model in results
    }

def format_table(metric="is_valid"):
    df = pd.DataFrame(reshuffle[metric])
    df = df.loc[:, ["gemini-2.5-flash-preview-05-20", "gemma-3-27b-it", "gemma-3-12b-it", "gemma-3-4b-it", "gemma-3-1b-it"]]
    return df.style.background_gradient(cmap="RdYlGn", vmin=0.2, vmax=1).format("{:.2f}")


for metric in ["is_valid", "compliance", "correctness"]:
    print("-"*100)
    print(metric)
    display(format_table(metric))
    print("\n")

----------------------------------------------------------------------------------------------------
is_valid


,gemini-2.5-flash-preview-05-20,gemma-3-27b-it,gemma-3-12b-it,gemma-3-4b-it,gemma-3-1b-it
1-rotowire,0.90,0.90,0.90,0.90,0.81
2-wiki_bio,1.00,0.69,0.73,0.83,0.50
3-few_nerd,0.93,0.90,0.90,0.90,0.00
4-TOPv1,0.93,0.90,0.86,0.90,0.01
5-api_bank,0.99,1.00,1.00,0.86,0.19
6-reasoning/GSM8K,0.89,0.92,0.84,0.79,0.69
6-reasoning/last_letter,0.90,0.90,0.90,0.90,0.98




----------------------------------------------------------------------------------------------------
compliance


,gemini-2.5-flash-preview-05-20,gemma-3-27b-it,gemma-3-12b-it,gemma-3-4b-it,gemma-3-1b-it
1-rotowire,1.00,0.99,0.98,0.94,0.88
2-wiki_bio,0.97,0.97,0.97,0.96,0.82
3-few_nerd,1.00,1.00,0.99,1.00,0.75
4-TOPv1,0.95,0.83,0.77,0.77,0.62
5-api_bank,0.99,1.00,1.00,0.98,0.21
6-reasoning/GSM8K,1.00,1.00,1.00,0.97,0.86
6-reasoning/last_letter,1.00,1.00,1.00,1.00,1.00




----------------------------------------------------------------------------------------------------
correctness


,gemini-2.5-flash-preview-05-20,gemma-3-27b-it,gemma-3-12b-it,gemma-3-4b-it,gemma-3-1b-it
1-rotowire,0.54,0.50,0.54,0.49,0.40
2-wiki_bio,0.33,0.33,0.32,0.35,0.24
3-few_nerd,0.60,0.39,0.40,0.43,0.23
4-TOPv1,0.64,0.60,0.50,0.51,0.25
5-api_bank,0.92,0.82,0.81,0.80,0.32
6-reasoning/GSM8K,0.93,0.91,0.89,0.84,0.32
6-reasoning/last_letter,0.98,0.84,0.81,0.17,0.00
